<a href="https://colab.research.google.com/github/vifirsanova/empi/blob/main/AI/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Обучить gensim W2V с помощью генеративной версии датасета ASD QA

In [1]:
!pip install gensim

In [2]:
from gensim.models import Word2Vec

Здесь нужны только тексты, без мета-данных

In [3]:
!wget -O data.json https://raw.githubusercontent.com/vifirsanova/private_data/main/original.json?token=GHSAT0AAAAAACM56C3GX5KV3PVI42EVKOR6ZQXXPSQ

--2024-04-11 10:30:19--  https://raw.githubusercontent.com/vifirsanova/private_data/main/original.json?token=GHSAT0AAAAAACM56C3GX5KV3PVI42EVKOR6ZQXXPSQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2909948 (2.8M) [text/plain]
Saving to: ‘data.json’

data.json           100%[===================>]   2.77M  --.-KB/s    in 0.07s   

2024-04-11 10:30:20 (39.8 MB/s) - ‘data.json’ saved [2909948/2909948]



In [4]:
import json

with open('data.json', 'r') as f:
  data = json.load(f)

In [6]:
textual_data = ''

for title in data['data']:
  for elem in title['paragraphs']:
    textual_data += elem['context']
    textual_data += '\n'
    for qa in elem['qas']:
      textual_data += qa['question']
      textual_data += '\n'
      textual_data += qa['answers'][0]['text']
      textual_data += '\n'

textual_data

'Этот текст подготовила группа аутичных взрослых, которые считают, что аналогичные статьи для родителей дают неточное представление об аутизме, в частности, они умалчивают о взрослых людях с расстройствами аутичного спектра. Аутизм является пожизненным состоянием, но по описаниям в СМИ создается ощущение, что он влияет только на маленьких детей. Аутичные дети вырастают и становятся аутичными взрослыми со своими специфическими потребностями, они также нуждаются в поддержке и имеют собственное мнение.\nО чем молчат статьи?\nони умалчивают о взрослых людях с расстройствами аутичного спектра.\nВ статьях можно найти любую информацию?\nони умалчивают о взрослых людях с расстройствами аутичного спектра.\nАутизм лечится?\nАутизм является пожизненным состоянием\nАутизм проходит?\nАутизм является пожизненным состоянием\nАутизм бывает только у детей?\nАутизм является пожизненным состоянием, но по описаниям в СМИ создается ощущение, что он влияет только на маленьких детей.\nАутизм - это детское за

In [29]:
import re

def tokenize(text):
    tokenized = []
    texts = text.split('\n')
    for t in texts:
      t = re.sub(r'[^\w\s]', ' ', t)
      tokenized.append(t.lower().split())
    return tokenized

sentences = tokenize(textual_data)
sentences

[['этот',
  'текст',
  'подготовила',
  'группа',
  'аутичных',
  'взрослых',
  'которые',
  'считают',
  'что',
  'аналогичные',
  'статьи',
  'для',
  'родителей',
  'дают',
  'неточное',
  'представление',
  'об',
  'аутизме',
  'в',
  'частности',
  'они',
  'умалчивают',
  'о',
  'взрослых',
  'людях',
  'с',
  'расстройствами',
  'аутичного',
  'спектра',
  'аутизм',
  'является',
  'пожизненным',
  'состоянием',
  'но',
  'по',
  'описаниям',
  'в',
  'сми',
  'создается',
  'ощущение',
  'что',
  'он',
  'влияет',
  'только',
  'на',
  'маленьких',
  'детей',
  'аутичные',
  'дети',
  'вырастают',
  'и',
  'становятся',
  'аутичными',
  'взрослыми',
  'со',
  'своими',
  'специфическими',
  'потребностями',
  'они',
  'также',
  'нуждаются',
  'в',
  'поддержке',
  'и',
  'имеют',
  'собственное',
  'мнение'],
 ['о', 'чем', 'молчат', 'статьи'],
 ['они',
  'умалчивают',
  'о',
  'взрослых',
  'людях',
  'с',
  'расстройствами',
  'аутичного',
  'спектра'],
 ['в', 'статьях', 'мож

In [30]:
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2, workers=2)

In [31]:
model.save('word2vec_model.bin')

In [32]:
model_path = 'word2vec_model.bin'
model = Word2Vec.load(model_path)

word = 'аутизм'
model.wv.most_similar(word)

[('расстройство', 0.9978697299957275),
 ('всегда', 0.9976277351379395),
 ('вызывает', 0.9975504279136658),
 ('обычно', 0.9973691701889038),
 ('развивается', 0.9973104596138),
 ('состояние', 0.9972131252288818),
 ('проблема', 0.9971809387207031),
 ('девочка', 0.9971196055412292),
 ('родителям', 0.9969417452812195),
 ('тревожность', 0.9969096183776855)]

Сравним с готовыми векторами из НКРЯ

In [17]:
import gensim.downloader as api
api.info()

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [18]:
model_rus = api.load('word2vec-ruscorpora-300')

[==================================================] 100.0% 198.8/198.8MB downloaded


Здесь нет нужной нам информации - откзаываемся от них

In [28]:
model_rus.most_similar("аутизм")

KeyError: "Key 'о' not present in vocabulary"